# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 6451, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 6451 (delta 0), reused 0 (delta 0), pack-reused 6450 (from 2)
Receiving objects: 100% (6451/6451), 404.15 MiB | 19.11 MiB/s, done.
Resolving deltas: 100% (3575/3575), done.
Updating files: 100% (498/498), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 31.5 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprec

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithKFCV/SLIM'

np.random.seed(42)

## Import the repository **RECsys_Challenge2024**

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'SLIM_BPR',
    'n_folds': 5,
    'metric': 'MAP',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SLIM_BPR_MAP.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/working/RECsys_Challenge2024/Dataset/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/working/RECsys_Challenge2024/Dataset/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_k_folds import split_train_k_folds

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

folds = split_train_k_folds(URM_all, k=config['n_folds'])

In [12]:
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python

def objective_function_SLIM_BPR(optuna_trial):

    full_hyperp = {
                    "topK": optuna_trial.suggest_int("topK", 0, 500),
                    "epochs": optuna_trial.suggest_int("epochs", 1, 50),
                    "lambda_i": optuna_trial.suggest_float("lambda_i", 1e-5, 1e-1, log=True),
                    "lambda_j": optuna_trial.suggest_float("lambda_j", 1e-5, 1e-1, log=True),
                    "learning_rate": optuna_trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
                }
    
    validation_results = []
    
    for fold_idx, (URM_train_fold, URM_validation_fold) in enumerate(folds):
    
        recommender_instance = SLIM_BPR_Python(URM_train_fold)
        recommender_instance.fit(**full_hyperp)
        
        evaluator = EvaluatorHoldout(URM_validation_fold, cutoff_list=[10])
        result_df, _ = evaluator.evaluateRecommender(recommender_instance)
        
        validation_results.append(result_df.loc[10]["MAP"])
    
    return np.mean(validation_results)

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_SLIM_BPR, n_trials=20)

[I 2025-01-05 20:31:57,433] Using an existing study with name 'hyperparameters_tuning_SLIM_BPR_MAP' instead of creating a new one.


Epoch 1, Iteration 35736 in 10.66 seconds. Samples per second 3352.02
Epoch 2, Iteration 35736 in 7.71 seconds. Samples per second 4634.21
Epoch 3, Iteration 35736 in 6.21 seconds. Samples per second 5754.69
Epoch 4, Iteration 35736 in 5.31 seconds. Samples per second 6724.24
Epoch 5, Iteration 35736 in 4.55 seconds. Samples per second 7849.78
Epoch 6, Iteration 35736 in 4.18 seconds. Samples per second 8547.33
Epoch 7, Iteration 35736 in 3.94 seconds. Samples per second 9081.15
Epoch 8, Iteration 35736 in 3.70 seconds. Samples per second 9663.26
Epoch 9, Iteration 35736 in 3.54 seconds. Samples per second 10090.82
Epoch 10, Iteration 35736 in 3.38 seconds. Samples per second 10585.69
Epoch 11, Iteration 35736 in 3.33 seconds. Samples per second 10727.70
Epoch 12, Iteration 35736 in 3.27 seconds. Samples per second 10943.69
Epoch 13, Iteration 35736 in 3.23 seconds. Samples per second 11057.80
Epoch 14, Iteration 35736 in 3.13 seconds. Samples per second 11429.48
Epoch 15, Iteration 35

[I 2025-01-05 21:05:48,576] Trial 15 finished with value: 0.03287386861548083 and parameters: {'topK': 160, 'epochs': 48, 'lambda_i': 6.0524476121265005e-05, 'lambda_j': 0.005434123937595034, 'learning_rate': 0.016465850478772966}. Best is trial 1 with value: 0.03301220945830492.


Epoch 1, Iteration 35736 in 9.56 seconds. Samples per second 3738.00
Epoch 2, Iteration 35736 in 6.92 seconds. Samples per second 5163.43
Epoch 3, Iteration 35736 in 5.85 seconds. Samples per second 6110.36
Epoch 4, Iteration 35736 in 4.97 seconds. Samples per second 7197.29
Epoch 5, Iteration 35736 in 4.47 seconds. Samples per second 8000.00
Epoch 6, Iteration 35736 in 4.07 seconds. Samples per second 8780.66
Epoch 7, Iteration 35736 in 3.83 seconds. Samples per second 9329.80
Epoch 8, Iteration 35736 in 3.58 seconds. Samples per second 9994.66
Epoch 9, Iteration 35736 in 3.52 seconds. Samples per second 10162.27
Epoch 10, Iteration 35736 in 3.53 seconds. Samples per second 10133.50
Epoch 11, Iteration 35736 in 3.39 seconds. Samples per second 10552.63
Epoch 12, Iteration 35736 in 3.38 seconds. Samples per second 10559.99
Epoch 13, Iteration 35736 in 3.36 seconds. Samples per second 10639.20
Epoch 14, Iteration 35736 in 3.30 seconds. Samples per second 10830.37
Epoch 15, Iteration 357

[I 2025-01-05 21:43:09,783] Trial 16 finished with value: 0.03031257409082953 and parameters: {'topK': 170, 'epochs': 50, 'lambda_i': 0.0011642745297896705, 'lambda_j': 0.0035065348523774043, 'learning_rate': 0.0009613859600652928}. Best is trial 1 with value: 0.03301220945830492.


...

[I 2025-01-06 06:28:59,148] Trial 32 finished with value: 0.04002380472623519 and parameters: {'topK': 22, 'epochs': 47, 'lambda_i': 0.020263504716694077, 'lambda_j': 0.00047570775780803477, 'learning_rate': 0.06354402019361458}. Best is trial 25 with value: 0.04090163381949502.


Epoch 1, Iteration 35736 in 9.96 seconds. Samples per second 3589.39
Epoch 2, Iteration 35736 in 7.22 seconds. Samples per second 4952.99
Epoch 3, Iteration 35736 in 5.93 seconds. Samples per second 6028.90
Epoch 4, Iteration 35736 in 5.13 seconds. Samples per second 6960.33
Epoch 5, Iteration 35736 in 4.66 seconds. Samples per second 7668.21
Epoch 6, Iteration 35736 in 4.19 seconds. Samples per second 8519.31
Epoch 7, Iteration 35736 in 3.95 seconds. Samples per second 9044.94
Epoch 8, Iteration 35736 in 3.83 seconds. Samples per second 9337.16
Epoch 9, Iteration 35736 in 3.80 seconds. Samples per second 9404.12
Epoch 10, Iteration 35736 in 3.60 seconds. Samples per second 9940.03
Epoch 11, Iteration 35736 in 3.62 seconds. Samples per second 9880.76
Epoch 12, Iteration 35736 in 3.53 seconds. Samples per second 10132.96
Epoch 13, Iteration 35736 in 6.78 seconds. Samples per second 5272.63
Epoch 14, Iteration 35736 in 3.59 seconds. Samples per second 9959.00
Epoch 15, Iteration 35736 in

[I 2025-01-06 07:03:44,249] Trial 33 finished with value: 0.036198686946664484 and parameters: {'topK': 83, 'epochs': 40, 'lambda_i': 0.00586243074857972, 'lambda_j': 0.0004155651792405946, 'learning_rate': 0.07033949402587175}. Best is trial 25 with value: 0.04090163381949502.


Epoch 1, Iteration 35736 in 9.97 seconds. Samples per second 3583.60
Epoch 2, Iteration 35736 in 7.35 seconds. Samples per second 4859.25
Epoch 3, Iteration 35736 in 5.92 seconds. Samples per second 6039.98
Epoch 4, Iteration 35736 in 5.12 seconds. Samples per second 6986.20
Epoch 5, Iteration 35736 in 4.69 seconds. Samples per second 7612.90
Epoch 6, Iteration 35736 in 4.31 seconds. Samples per second 8292.41
Epoch 7, Iteration 35736 in 3.93 seconds. Samples per second 9087.58
Epoch 8, Iteration 35736 in 3.79 seconds. Samples per second 9420.32
Epoch 9, Iteration 35736 in 3.66 seconds. Samples per second 9768.90
Epoch 10, Iteration 35736 in 3.47 seconds. Samples per second 10299.62
Epoch 11, Iteration 35736 in 3.47 seconds. Samples per second 10295.58
Epoch 12, Iteration 35736 in 3.39 seconds. Samples per second 10538.69
Epoch 13, Iteration 35736 in 3.29 seconds. Samples per second 10874.11
Epoch 14, Iteration 35736 in 3.31 seconds. Samples per second 10799.77
Epoch 15, Iteration 3573

[I 2025-01-06 07:37:03,765] Trial 34 finished with value: 0.040810184455877536 and parameters: {'topK': 15, 'epochs': 40, 'lambda_i': 0.028589336237712655, 'lambda_j': 6.0107655642311166e-05, 'learning_rate': 0.0770883834960638}. Best is trial 25 with value: 0.04090163381949502.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = SLIM_BPR_Python(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

Epoch 1, Iteration 35736 in 10.72 seconds. Samples per second 3333.20
Epoch 2, Iteration 35736 in 7.77 seconds. Samples per second 4599.89
Epoch 3, Iteration 35736 in 6.09 seconds. Samples per second 5871.09
Epoch 4, Iteration 35736 in 5.24 seconds. Samples per second 6816.85
Epoch 5, Iteration 35736 in 4.68 seconds. Samples per second 7641.55
Epoch 6, Iteration 35736 in 4.17 seconds. Samples per second 8566.03
Epoch 7, Iteration 35736 in 3.91 seconds. Samples per second 9136.54
Epoch 8, Iteration 35736 in 3.67 seconds. Samples per second 9728.42
Epoch 9, Iteration 35736 in 3.70 seconds. Samples per second 9669.67
Epoch 10, Iteration 35736 in 3.42 seconds. Samples per second 10452.19
Epoch 11, Iteration 35736 in 3.33 seconds. Samples per second 10745.78
Epoch 12, Iteration 35736 in 3.34 seconds. Samples per second 10714.23
Epoch 13, Iteration 35736 in 3.27 seconds. Samples per second 10928.15
Epoch 14, Iteration 35736 in 3.25 seconds. Samples per second 10979.48
Epoch 15, Iteration 357

# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/working/RECsys_Challenge2024/Dataset/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_SLIM_BPR_MAP.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/best_params_SLIM_BPR_MAP.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/history_SLIM_BPR_MAP.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithKFCV/SLIM/SLIM_BPRRecommender/OptimizingMAP/Submission/submission_SLIM_BPR_MAP.csv' updated successfully.
